In [1]:
%run preamble_scripts.py

# Simulation study: the well specified case 2000 observations

In [2]:
n_sim, nobs, expo = 1000, 5000, 250
err_pnl, popsize = 3e5, 10000

In [3]:
# Setting the true model
f1, f2 = bs.loss_model("Weibull", ["k1", "β1"]), bs.loss_model("Lomax", ["α2", "σ2"])
f_true = bs.spliced_loss_model(f1, f2, "continuous")
parms_true = np.array([1/2, 1, 2.5, 3, 1.5])

f_true.set_ppf(), f_true.set_pdf(),f_true.set_cdf()

# We set the priority to the 90% quantile and the limit to the 0.99% quantile
P, L = f_true.ppf(parms_true, 0.9), f_true.ppf(parms_true, 0.99)
premiums = f_true.PP(parms_true), f_true.XOLP(parms_true, P, L)

%time PnLs = np.array(f_true.PnL(parms_true, P, L, expo, premiums, safety_loadings = [0.05, 0.05],  n_sim = err_pnl))
true_VaRs = [f_true.ppf(parms_true, prob) for prob in [0.95, 0.99, 0.995]]
true_cap = np.quantile(PnLs, [0.005, 0.01, 0.05])
true_cap

CPU times: user 38.4 s, sys: 36 ms, total: 38.4 s
Wall time: 38.4 s


array([-176.26278572, -128.95343056,  -54.59176095])

We are interested in the estimations of the extreme quantile of the claim size distribution (of order 0.95, 0.99, 0.995) and the quantile of the aggregate losses over one year with a XOL reinsurance agreement

In [4]:
Xs = [f_true.sample(parms_true, nobs) for k in range(n_sim)]
def fit_one(k, f, f_name,  prior, X):
    trace, log_marg, DIC, WAIC = bs.smc(X, f, popsize, prior, verbose = False)
    PnLs = np.array(f.PnL(trace.mean().values, P, L, expo, premiums, safety_loadings = [0.05, 0.05],  n_sim = err_pnl))
    cap = np.quantile(PnLs, [0.005, 0.01, 0.05])
    return([k, f_name , nobs, log_marg, trace["γ"].mean(), parms_true[-1] ] + cap.tolist() + true_cap.tolist())
def fit_all(k):
    print("simulation #"+ str(k))
    col_names = ["sim", "model_name", "nobs", "log_marg", "γ_map", "γ_true", "cap005", "cap01", "cap05", "cap005_true", "cap01_true", "cap05_true" ]
    df_temp = pd.DataFrame([fit_one(k, fs[i], f_names[i], prior_spliced_model[i], Xs[k]) for i in range(len(fs))], columns = col_names)
    df_temp['posterior_probability'] = np.exp(df_temp['log_marg'].values - np.max(df_temp['log_marg'].values)) / \
    np.sum(np.exp(df_temp['log_marg'].values - np.max(df_temp['log_marg'].values)))  
    γ_BMA = np.dot(df_temp['posterior_probability'].values, df_temp['γ_map'])
    cap_bma = np.dot(df_temp['posterior_probability'].values, df_temp[["cap005", "cap01", "cap05"]].values)
    BMA_df = pd.DataFrame([[k, "BMA" , nobs, None, γ_BMA, parms_true[-1] ] + cap_bma.tolist() + true_cap.tolist()],
            columns = col_names)
    df_temp = pd.concat([df_temp, BMA_df])
    return(df_temp)
%time res = Parallel(n_jobs= 50)(delayed(fit_all)(k) for k in range(n_sim))
pd.concat(res).to_csv("../../Data/Simulations/simu_wellspec_"+str(nobs)+".csv", sep=',')

/home/pgoffard/miniconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


CPU times: user 3min 4s, sys: 54.8 s, total: 3min 59s
Wall time: 1d 14h 22min 51s


In [5]:
pd.concat(res)

,sim,model_name,nobs,log_marg,γ_map,γ_true,cap005,cap01,cap05,cap005_true,cap01_true,cap05_true,posterior_probability
0,0,Weibull_Log-Logistic,5000,-3823.66223,2.566335,1.5,-393.316015,-257.237486,-87.394002,-176.262786,-128.953431,-54.591761,1.999001e-01
1,0,Weibull_Lomax,5000,-3822.480563,2.044409,1.5,-237.518967,-165.314228,-65.327566,-176.262786,-128.953431,-54.591761,6.516350e-01
2,0,Weibull_Inverse-Weibull,5000,-3823.959699,2.857912,1.5,-430.538805,-282.875339,-96.322515,-176.262786,-128.953431,-54.591761,1.484649e-01
3,0,Inverse-Weibull_Log-Logistic,5000,-3959.310756,0.180976,1.5,-2581.792724,-1575.059482,-476.372487,-176.262786,-128.953431,-54.591761,2.451357e-60
4,0,Inverse-Weibull_Lomax,5000,-3926.570876,0.123042,1.5,-465.541002,-318.000560,-118.332817,-176.262786,-128.953431,-54.591761,4.056539e-46
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,999,Weibull_Inverse-Weibull,5000,-4141.890438,3.132476,1.5,-305.768419,-202.954292,-74.326596,-176.262786,-128.953431,-54.591761,2.190327e-03
3,999,Inverse-Weibull_Log-Logistic,5000,-4313.464236,0.149965,1.5,-2981.786340,-1780.836722,-534.771521,-176.262786,-128.953431,-54.591761,6.713596e-78
4,999,Inverse-Weibull_Lomax,5000,-4268.058221,0.091195,1.5,-469.518385,-320.085106,-123.024368,-176.262786,-128.953431,-54.591761,3.519952e-58
5,999,Inverse-Weibull_Inverse-Weibull,5000,-4333.290489,1.856897,1.5,-417.546329,-273.370534,-91.881831,-176.262786,-128.953431,-54.591761,1.646354e-86
